In [1]:
import tensorflow as tf
print(tf.__version__)

# !pip install -q tensorflow-datasets

2.2.0


In [4]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Shuffling and writing examples to /Users/vishwasmore/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVYYTCC/imdb_reviews-train.tfrecord


Shuffling and writing examples to /Users/vishwasmore/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVYYTCC/imdb_reviews-test.tfrecord


Shuffling and writing examples to /Users/vishwasmore/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVYYTCC/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /Users/vishwasmore/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [6]:
imdb, info

({'test': <PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>,
  'train': <PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>,
  'unsupervised': <PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>},
 tfds.core.DatasetInfo(
     name='imdb_reviews',
     version=1.0.0,
     description='Large Movie Review Dataset.
 This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
     homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
     features=FeaturesDict({
         'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
         'text': Text(shape=(), dtype=tf.string),
     }),
     total_num_examples=100000,
     splits={
         'test': 25000,
         'train': 25000,
         'unsupervised': 50000,
     },
     supervised_key

In [7]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']


In [38]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s,l in train_data:
    training_sentences.append(s.numpy().decode('utf8'))
    training_labels.append(l.numpy())
    
for s,l in test_data:
    testing_sentences.append(s.numpy().decode('utf8'))
    testing_labels.append(l.numpy())

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [39]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"

In [40]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [41]:
# word_index.items()
padded[3]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,   12,    7,
          2,  241,    5,   20,   16,    4, 8776, 2707, 2653,   52,    2,
        358,    5,    2,  180,   68,  138, 1401,   17,   92,  203,  968,
         15,   23,    1,   81,    4,  192, 3109, 3037,    3,    1,   16,
          4,  376,    5,  632,  387,  352,   37, 6355,    3, 5559, 1928,
         15,  208, 8596, 3433,    2,  112,  365,   48,   24,   55,    1,
          6, 1657,   55, 1818, 4312,   41,    4, 2263,    3, 1908,    1,
        141,  160,  780,  111,   31,   92,  116,    4,  221,   20,    9,
        173,  279,    3,   29,   13, 1012,    2, 2814,   10, 1985],
      dtype=int32)

In [42]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i,'?') for i in text])

print(decode_review(padded[3]))

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received


In [43]:
print(training_sentences[3])

This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [36]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [44]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 1s 2ms/step - loss: 0.4884 - accuracy: 0.7466 - val_loss: 0.3398 - val_accuracy: 0.8512
Epoch 2/10
782/782 [==============================] - 1s 2ms/step - loss: 0.2421 - accuracy: 0.9052 - val_loss: 0.3834 - val_accuracy: 0.8325
Epoch 3/10
782/782 [==============================] - 1s 2ms/step - loss: 0.0969 - accuracy: 0.9743 - val_loss: 0.4649 - val_accuracy: 0.8250
Epoch 4/10
782/782 [==============================] - 1s 2ms/step - loss: 0.0250 - accuracy: 0.9966 - val_loss: 0.5312 - val_accuracy: 0.8273
Epoch 5/10
782/782 [==============================] - 1s 2ms/step - loss: 0.0070 - accuracy: 0.9993 - val_loss: 0.5997 - val_accuracy: 0.8267
Epoch 6/10
782/782 [==============================] - 1s 2ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.6554 - val_accuracy: 0.8279
Epoch 7/10
782/782 [==============================] - 1s 2ms/step - loss: 8.8091e-04 - accuracy: 1.0000 - val_loss: 0.7015 - val_accuracy: 0.8288
Ep

In [45]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [47]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")

out_v.close()
out_m.close()

In [48]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
